In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np


In [2]:
#Write name of location of the folder 
path_directory="/Users/lauradebarre/Documents/Temporary/Water_domestic_Human_health_v1.1/"

# Fate factor

In [3]:
FF_shape=gpd.read_file(path_directory+"/data/shape_bas34.shp")
FF_shape

,BAS34S_ID,BASIN0_ID,Area,TOT_CU,geometry
0,1,1,3.394890e+08,0.0,"POLYGON ((-38.00003 83.49997, -38.50003 83.499..."
1,2,2,3.394890e+08,0.0,"POLYGON ((-37.50003 83.49997, -38.00003 83.499..."
2,3,3,3.394890e+08,0.0,"POLYGON ((-37.00003 83.49997, -37.50003 83.499..."
3,4,4,3.394890e+08,0.0,"POLYGON ((-36.50003 83.49997, -37.00003 83.499..."
4,5,5,3.394890e+08,0.0,"POLYGON ((-36.00003 83.49997, -36.50003 83.499..."
...,...,...,...,...,...
11045,66892,66892,1.769990e+09,4527.0,"POLYGON ((-66.00003 -55.50003, -66.50003 -55.5..."
11046,66893,66893,1.747850e+09,229.0,"POLYGON ((-69.00003 -56.00003, -69.50003 -56.0..."
11047,66894,66894,1.747850e+09,253270.0,"POLYGON ((-68.00003 -56.00003, -68.50003 -56.0..."
11048,66895,66895,1.747850e+09,118.0,"POLYGON ((-67.50003 -56.00003, -68.00003 -56.0..."


In [4]:
# crop FF_shape to necessary data only
columns_to_drop = ["BASIN0_ID", "Area","TOT_CU"]
FF_shaped = FF_shape.drop(columns=columns_to_drop)
FF_shaped

,BAS34S_ID,geometry
0,1,"POLYGON ((-38.00003 83.49997, -38.50003 83.499..."
1,2,"POLYGON ((-37.50003 83.49997, -38.00003 83.499..."
2,3,"POLYGON ((-37.00003 83.49997, -37.50003 83.499..."
3,4,"POLYGON ((-36.50003 83.49997, -37.00003 83.499..."
4,5,"POLYGON ((-36.00003 83.49997, -36.50003 83.499..."
...,...,...
11045,66892,"POLYGON ((-66.00003 -55.50003, -66.50003 -55.5..."
11046,66893,"POLYGON ((-69.00003 -56.00003, -69.50003 -56.0..."
11047,66894,"POLYGON ((-68.00003 -56.00003, -68.50003 -56.0..."
11048,66895,"POLYGON ((-67.50003 -56.00003, -68.00003 -56.0..."


In [5]:
#importing dataset to calculate FF, and doing the calculations 

fate_text = pd.read_csv(path_directory+'/data/avail_wh_monthly_dont_scarcity_corrected.csv', sep=";", decimal=',')

fate_text2=fate_text[["Bas34_ID","TOTAL_avail_wh_monthly_m3","dom"]] #selecting only interesting columns

#avoiding mixed data types
fate_text2["dom"] = pd.to_numeric(fate_text2["dom"], errors='coerce')
fate_text2["TOTAL_avail_wh_monthly_m3"] = pd.to_numeric(fate_text2["TOTAL_avail_wh_monthly_m3"], errors='coerce')


#calculating Scarcity, but not limited (yet) : F as the ratio of dom over availability
fate_text2["Scarcity_not_limited"] = np.where(  
    fate_text2["TOTAL_avail_wh_monthly_m3"] == 0,
    np.where(fate_text2["dom"] > 0, "infinite", np.nan),
    fate_text2["dom"] / fate_text2["TOTAL_avail_wh_monthly_m3"]
)

# Calculate FF as the ratio of dom over availability setting conditions for values of 0 or > 1
result=fate_text2["dom"]/fate_text2["TOTAL_avail_wh_monthly_m3"]
fate_text2["Scarcity"] = np.where(
    fate_text2["TOTAL_avail_wh_monthly_m3"] == 0,
    np.where(fate_text2["dom"] > 0, 1, np.nan),
    np.where(result > 1, 1, result)
)

#fate_text2 = fate_text2.dropna(how='all')
fate_text2.sort_values(by="Scarcity", ascending=False)

<ipython-input-5-d4a0b7dd7af5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fate_text2["dom"] = pd.to_numeric(fate_text2["dom"], errors='coerce')
<ipython-input-5-d4a0b7dd7af5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fate_text2["TOTAL_avail_wh_monthly_m3"] = pd.to_numeric(fate_text2["TOTAL_avail_wh_monthly_m3"], errors='coerce')
<ipython-input-5-d4a0b7dd7af5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,Bas34_ID,TOTAL_avail_wh_monthly_m3,dom,Scarcity_not_limited,Scarcity
8426,50459.0,20260000.0,33819137.0,1.6692565153010859,1.0
6997,40454.0,0.0,486871.0,infinite,1.0
6847,39049.0,0.0,13765.0,infinite,1.0
6551,36816.0,40000.0,287358.0,7.18395,1.0
6875,39259.0,0.0,31333339.0,infinite,1.0
...,...,...,...,...,...
6324,35441.0,0.0,0.0,nan,NaN
6325,35442.0,0.0,0.0,nan,NaN
6326,35443.0,0.0,0.0,nan,NaN
7408,44288.0,0.0,0.0,nan,NaN


In [6]:
# merging pgeo dataframe and FF dataframe in a pgeodataframe FF_with_geographies

# Rename 'Bas34_ID' column in 'fate_text2' to match 'BAS34S_ID' in 'FF_shaped'
fate_text2.rename(columns={'Bas34_ID': 'BAS34S_ID'}, inplace=True)

# Merging the two DataFrames based on 'BAS34S_ID'
FF_with_geographies = pd.merge(FF_shaped, fate_text2, on='BAS34S_ID', how='left')
FF_with_geographies

FF_with_geographies.to_csv(path_directory+"/results/FF/FF_with_geographies.csv")
FF_with_geographies.to_file(path_directory+"/results/FF/FF_with_geographies.shp")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-6-5812f7d29dc5>:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  FF_with_geographies.to_file(path_directory+"/results/FF/FF_with_geographies.shp")


# Exposure factor

In [7]:
#import pgeo dataframe with country borders
Countries_shape=gpd.read_file(path_directory+"/data/Countries_Combined_with_associated_territories.shp")
Countries_shape

,OBJECTID,ISO3CD,ROMNAM,MAP_LABEL,MAP_COLOR,STATUS,M49Code,geometry
0,1,AFG,Afghanistan,Afghanistan,AFG,Member State,4,"POLYGON ((71.00146 38.47707, 71.00709 38.47256..."
1,2,AGO,Angola,Angola,AGO,Member State,24,"MULTIPOLYGON (((14.07200 -5.86461, 14.08028 -5..."
2,3,ALB,Albania,Albania,ALB,Member State,8,"MULTIPOLYGON (((19.74975 42.62830, 19.74977 42..."
3,4,AND,Andorra,Andorra,AND,Member State,20,"POLYGON ((1.55249 42.65408, 1.55801 42.65205, ..."
4,5,ARE,United Arab Emirates,United Arab Emirates,ARE,Member State,784,"MULTIPOLYGON (((56.16162 26.05599, 56.16382 26..."
...,...,...,...,...,...,...,...,...
200,202,xxx,China/India,None,xxx,Undetermined,1021,"MULTIPOLYGON (((79.22455 33.23460, 79.22986 33..."
201,203,YEM,Yemen,Yemen,YEM,Member State,887,"MULTIPOLYGON (((52.75139 17.30417, 52.74321 17..."
202,204,ZAF,South Africa,South Africa,ZAF,Member State,710,"MULTIPOLYGON (((29.67351 -22.13859, 29.67541 -..."
203,205,ZMB,Zambia,Zambia,ZMB,Member State,894,"POLYGON ((30.47854 -8.27828, 30.48844 -8.28598..."


In [8]:
GNI_per_cap_2022=pd.read_excel(path_directory+"/data/GNI_per_capita_2022.xls", sheet_name="Data") #importing dataset with 2022 GNI per capita
GNI_per_cap_2022=GNI_per_cap_2022[["Country Name","Country Code","Indicator Name","2022b"]] #selecting interesting columns
GNI_per_cap_2022

,Country Name,Country Code,Indicator Name,2022b
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",29460.000000
1,Africa Eastern and Southern,AFE,"GNI per capita, Atlas method (current US$)",1542.260985
2,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",390.000000
3,Africa Western and Central,AFW,"GNI per capita, Atlas method (current US$)",1779.431390
4,Angola,AGO,"GNI per capita, Atlas method (current US$)",1900.000000
...,...,...,...,...
261,Kosovo,XKX,"GNI per capita, Atlas method (current US$)",5590.000000
262,"Yemen, Rep.",YEM,"GNI per capita, Atlas method (current US$)",840.000000
263,South Africa,ZAF,"GNI per capita, Atlas method (current US$)",6780.000000
264,Zambia,ZMB,"GNI per capita, Atlas method (current US$)",1170.000000


In [9]:
#calculate the Exposure Factor - XF 

year_of_reference="2022"

#conditions for exposure calculation 
lim_low_income=1046 
lim_high_income=12695
def adaptation_capacity(x):
    return(x/(lim_high_income-lim_low_income) + (-lim_low_income)/(lim_high_income-lim_low_income))


dtf=GNI_per_cap_2022
# Define the conditions for country income classification 
condition_1 = (dtf['2022b'] == 0)
condition_2 = (dtf['2022b'] > lim_high_income)
condition_3 = (dtf['2022b'] < lim_low_income)
condition_4 = dtf['2022b'].between(lim_low_income, lim_high_income)

value_1 = ""
value_2 = 1
value_3 = 0
value_4 = (adaptation_capacity(dtf['2022b']))
           
# Create the "AC" column based on the conditions
dtf['AC'] = np.select([condition_1, condition_2, condition_3, condition_4],
                     [value_1, value_2, value_3, value_4], default=np.nan)

dtf['XF']=1-pd.to_numeric(dtf['AC'], errors='coerce')


dtf.to_csv(path_directory+"/results/XF/XF.csv")
dtf


,Country Name,Country Code,Indicator Name,2022b,AC,XF
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",29460.000000,1,0.000000
1,Africa Eastern and Southern,AFE,"GNI per capita, Atlas method (current US$)",1542.260985,0.042601166160215254,0.957399
2,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",390.000000,0,1.000000
3,Africa Western and Central,AFW,"GNI per capita, Atlas method (current US$)",1779.431390,0.06296088850958331,0.937039
4,Angola,AGO,"GNI per capita, Atlas method (current US$)",1900.000000,0.07331101382092883,0.926689
...,...,...,...,...,...,...
261,Kosovo,XKX,"GNI per capita, Atlas method (current US$)",5590.000000,0.39007640140784616,0.609924
262,"Yemen, Rep.",YEM,"GNI per capita, Atlas method (current US$)",840.000000,0,1.000000
263,South Africa,ZAF,"GNI per capita, Atlas method (current US$)",6780.000000,0.49223109279766497,0.507769
264,Zambia,ZMB,"GNI per capita, Atlas method (current US$)",1170.000000,0.010644690531376094,0.989355


In [10]:
# create pgeodataframe with XF 

XF_shape=pd.merge(Countries_shape, dtf.rename(columns={"Country Code":"ISO3CD"}), on="ISO3CD", how='left')
XF_shape

,OBJECTID,ISO3CD,ROMNAM,MAP_LABEL,MAP_COLOR,STATUS,M49Code,geometry,Country Name,Indicator Name,2022b,AC,XF
0,1,AFG,Afghanistan,Afghanistan,AFG,Member State,4,"POLYGON ((71.00146 38.47707, 71.00709 38.47256...",Afghanistan,"GNI per capita, Atlas method (current US$)",390.0,0,1.000000
1,2,AGO,Angola,Angola,AGO,Member State,24,"MULTIPOLYGON (((14.07200 -5.86461, 14.08028 -5...",Angola,"GNI per capita, Atlas method (current US$)",1900.0,0.07331101382092883,0.926689
2,3,ALB,Albania,Albania,ALB,Member State,8,"MULTIPOLYGON (((19.74975 42.62830, 19.74977 42...",Albania,"GNI per capita, Atlas method (current US$)",6770.0,0.49137265001287667,0.508627
3,4,AND,Andorra,Andorra,AND,Member State,20,"POLYGON ((1.55249 42.65408, 1.55801 42.65205, ...",Andorra,"GNI per capita, Atlas method (current US$)",46530.0,1,0.000000
4,5,ARE,United Arab Emirates,United Arab Emirates,ARE,Member State,784,"MULTIPOLYGON (((56.16162 26.05599, 56.16382 26...",United Arab Emirates,"GNI per capita, Atlas method (current US$)",48950.0,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,202,xxx,China/India,None,xxx,Undetermined,1021,"MULTIPOLYGON (((79.22455 33.23460, 79.22986 33...",NaN,NaN,NaN,NaN,NaN
201,203,YEM,Yemen,Yemen,YEM,Member State,887,"MULTIPOLYGON (((52.75139 17.30417, 52.74321 17...","Yemen, Rep.","GNI per capita, Atlas method (current US$)",840.0,0,1.000000
202,204,ZAF,South Africa,South Africa,ZAF,Member State,710,"MULTIPOLYGON (((29.67351 -22.13859, 29.67541 -...",South Africa,"GNI per capita, Atlas method (current US$)",6780.0,0.49223109279766497,0.507769
203,205,ZMB,Zambia,Zambia,ZMB,Member State,894,"POLYGON ((30.47854 -8.27828, 30.48844 -8.28598...",Zambia,"GNI per capita, Atlas method (current US$)",1170.0,0.010644690531376094,0.989355


# EF

In [11]:
#import EF values per country
EF_values=pd.read_excel(path_directory+'/data/EF (pi values from JMP2020).xlsx', sheet_name="Country scale EF corrected", skiprows=1)
EF_values.sort_values(by="EF", ascending=False)
EF_values.to_csv(path_directory+'/data/EF_country_values.csv')

In [12]:
EF_values["EF"].describe()

count    231.000000
mean       0.000253
std        0.000184
min        0.000075
25%        0.000088
50%        0.000198
75%        0.000385
max        0.000872
Name: EF, dtype: float64

# CF computation - subriver basin scale

In [13]:
# Perform the spatial intersection
intersection_FF_country = gpd.overlay(FF_with_geographies, XF_shape, how='intersection')
intersection_FF_country

,BAS34S_ID,TOTAL_avail_wh_monthly_m3,dom,Scarcity_not_limited,Scarcity,OBJECTID,ISO3CD,ROMNAM,MAP_LABEL,MAP_COLOR,STATUS,M49Code,Country Name,Indicator Name,2022b,AC,XF,geometry
0,2,7.680000e+06,0.0,0.0,0.000000,49,DNK,Denmark,Denmark,DNK,Member State,208,Denmark,"GNI per capita, Atlas method (current US$)",73200.0,1,0.0,"POLYGON ((-37.88129 83.49997, -37.86035 83.500..."
1,4,1.491000e+07,0.0,0.0,0.000000,49,DNK,Denmark,Denmark,DNK,Member State,208,Denmark,"GNI per capita, Atlas method (current US$)",73200.0,1,0.0,"MULTIPOLYGON (((-36.79602 83.49997, -36.79509 ..."
2,5,1.603000e+07,0.0,0.0,0.000000,49,DNK,Denmark,Denmark,DNK,Member State,208,Denmark,"GNI per capita, Atlas method (current US$)",73200.0,1,0.0,"POLYGON ((-36.50003 83.49997, -36.50003 83.562..."
3,6,1.631000e+07,0.0,0.0,0.000000,49,DNK,Denmark,Denmark,DNK,Member State,208,Denmark,"GNI per capita, Atlas method (current US$)",73200.0,1,0.0,"POLYGON ((-36.00003 83.49997, -36.00003 83.558..."
4,7,1.592000e+07,0.0,0.0,0.000000,49,DNK,Denmark,Denmark,DNK,Member State,208,Denmark,"GNI per capita, Atlas method (current US$)",73200.0,1,0.0,"POLYGON ((-35.50003 83.49997, -35.50003 83.557..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12230,66602,2.316390e+09,332137.0,0.00014338561295809427,0.000143,135,NZL,New Zealand,New Zealand,NZL,Member State,554,New Zealand,"GNI per capita, Atlas method (current US$)",48460.0,1,0.0,"MULTIPOLYGON (((168.49997 -46.59490, 168.50013..."
12231,66603,3.548000e+08,18518.0,5.2192784667418265e-05,0.000052,135,NZL,New Zealand,New Zealand,NZL,Member State,554,New Zealand,"GNI per capita, Atlas method (current US$)",48460.0,1,0.0,"POLYGON ((168.99997 -46.50003, 169.49997 -46.5..."
12232,66604,5.949000e+07,1480.0,2.4878130778282065e-05,0.000025,135,NZL,New Zealand,New Zealand,NZL,Member State,554,New Zealand,"GNI per capita, Atlas method (current US$)",48460.0,1,0.0,"POLYGON ((169.49997 -46.50003, 169.71089 -46.5..."
12233,66623,3.108000e+08,1002.0,3.223938223938224e-06,0.000003,135,NZL,New Zealand,New Zealand,NZL,Member State,554,New Zealand,"GNI per capita, Atlas method (current US$)",48460.0,1,0.0,"MULTIPOLYGON (((167.49997 -47.19830, 167.50006..."


In [14]:
#cropping to interesting values only
FF_XF=intersection_FF_country[['BAS34S_ID', 'Scarcity','TOTAL_avail_wh_monthly_m3',"dom", 'ISO3CD', 'ROMNAM',
                         'XF', 'geometry']]
FF_XF

,BAS34S_ID,Scarcity,TOTAL_avail_wh_monthly_m3,dom,ISO3CD,ROMNAM,XF,geometry
0,2,0.000000,7.680000e+06,0.0,DNK,Denmark,0.0,"POLYGON ((-37.88129 83.49997, -37.86035 83.500..."
1,4,0.000000,1.491000e+07,0.0,DNK,Denmark,0.0,"MULTIPOLYGON (((-36.79602 83.49997, -36.79509 ..."
2,5,0.000000,1.603000e+07,0.0,DNK,Denmark,0.0,"POLYGON ((-36.50003 83.49997, -36.50003 83.562..."
3,6,0.000000,1.631000e+07,0.0,DNK,Denmark,0.0,"POLYGON ((-36.00003 83.49997, -36.00003 83.558..."
4,7,0.000000,1.592000e+07,0.0,DNK,Denmark,0.0,"POLYGON ((-35.50003 83.49997, -35.50003 83.557..."
...,...,...,...,...,...,...,...,...
12230,66602,0.000143,2.316390e+09,332137.0,NZL,New Zealand,0.0,"MULTIPOLYGON (((168.49997 -46.59490, 168.50013..."
12231,66603,0.000052,3.548000e+08,18518.0,NZL,New Zealand,0.0,"POLYGON ((168.99997 -46.50003, 169.49997 -46.5..."
12232,66604,0.000025,5.949000e+07,1480.0,NZL,New Zealand,0.0,"POLYGON ((169.49997 -46.50003, 169.71089 -46.5..."
12233,66623,0.000003,3.108000e+08,1002.0,NZL,New Zealand,0.0,"MULTIPOLYGON (((167.49997 -47.19830, 167.50006..."


In [15]:
#merging factors on the same file
FF_XF_EF=pd.merge(FF_XF, EF_values[['ISO3','EF','Region (World bank)','Region (United nations)', 'Region (WB/UN)', 'Region', 'Sub-region','Mean EF region WB/UN']].rename(columns={"ISO3":"ISO3CD"}), on='ISO3CD', how='inner')

#calculating CF
CF=FF_XF_EF
CF["CF"]=CF["Scarcity"]*CF["XF"]*CF["EF"]
CF["CF_regional_EF"]=CF["Scarcity"]*CF["XF"]*CF["Mean EF region WB/UN"]
CF

,BAS34S_ID,Scarcity,TOTAL_avail_wh_monthly_m3,dom,ISO3CD,ROMNAM,XF,geometry,EF,Region (World bank),Region (United nations),Region (WB/UN),Region,Sub-region,Mean EF region WB/UN,CF,CF_regional_EF
0,2,0.000000,7.680000e+06,0.0,DNK,Denmark,0.0,"POLYGON ((-37.88129 83.49997, -37.86035 83.500...",0.000084,Europe and Central Asia,Europe and Northern America,Europe and Central Asia,Europe,Northern Europe,0.000112,0.0,0.0
1,4,0.000000,1.491000e+07,0.0,DNK,Denmark,0.0,"MULTIPOLYGON (((-36.79602 83.49997, -36.79509 ...",0.000084,Europe and Central Asia,Europe and Northern America,Europe and Central Asia,Europe,Northern Europe,0.000112,0.0,0.0
2,5,0.000000,1.603000e+07,0.0,DNK,Denmark,0.0,"POLYGON ((-36.50003 83.49997, -36.50003 83.562...",0.000084,Europe and Central Asia,Europe and Northern America,Europe and Central Asia,Europe,Northern Europe,0.000112,0.0,0.0
3,6,0.000000,1.631000e+07,0.0,DNK,Denmark,0.0,"POLYGON ((-36.00003 83.49997, -36.00003 83.558...",0.000084,Europe and Central Asia,Europe and Northern America,Europe and Central Asia,Europe,Northern Europe,0.000112,0.0,0.0
4,7,0.000000,1.592000e+07,0.0,DNK,Denmark,0.0,"POLYGON ((-35.50003 83.49997, -35.50003 83.557...",0.000084,Europe and Central Asia,Europe and Northern America,Europe and Central Asia,Europe,Northern Europe,0.000112,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12209,66602,0.000143,2.316390e+09,332137.0,NZL,New Zealand,0.0,"MULTIPOLYGON (((168.49997 -46.59490, 168.50013...",0.000075,East Asia and Pacific,Oceania,East Asia and Pacific,Oceania,Australia and New Zealand,0.000253,0.0,0.0
12210,66603,0.000052,3.548000e+08,18518.0,NZL,New Zealand,0.0,"POLYGON ((168.99997 -46.50003, 169.49997 -46.5...",0.000075,East Asia and Pacific,Oceania,East Asia and Pacific,Oceania,Australia and New Zealand,0.000253,0.0,0.0
12211,66604,0.000025,5.949000e+07,1480.0,NZL,New Zealand,0.0,"POLYGON ((169.49997 -46.50003, 169.71089 -46.5...",0.000075,East Asia and Pacific,Oceania,East Asia and Pacific,Oceania,Australia and New Zealand,0.000253,0.0,0.0
12212,66623,0.000003,3.108000e+08,1002.0,NZL,New Zealand,0.0,"MULTIPOLYGON (((167.49997 -47.19830, 167.50006...",0.000075,East Asia and Pacific,Oceania,East Asia and Pacific,Oceania,Australia and New Zealand,0.000253,0.0,0.0


In [16]:
# Save as a shapefile
CF.to_file(path_directory+'/results/CF/CF.shp', driver='ESRI Shapefile')

#save as excel file
CF.to_excel(path_directory+"/results/CF/CF_per_subriver.xlsx")



<ipython-input-16-a78a5ceaeaf6>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  CF.to_file(path_directory+'/results/CF/CF.shp', driver='ESRI Shapefile')


In [17]:
CF["CF"].describe()

count    1.198700e+04
mean     1.686914e-06
std      2.032673e-05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.989959e-08
max      7.463064e-04
Name: CF, dtype: float64